# Horizontally and Vertically Text Detection

TextDetection function detect the text by vertical and horizontal orientation in the image.
Horizontal text has more width (wider) than height.
Vertical text has more height (taller) than width.

In [1]:
# Importing Libraries
import cv2
import pytesseract
from pytesseract import Output

In [2]:
# Function to detect text
# It returns detected horizontal and vertical detected text, box image, and OCR data
def TextDetection(img_path, config=r"--psm 11 --oem 3"):
    # Load the image
    img = cv2.imread(img_path)
    
    # Check if the image is loaded successfully
    if img is None:
        raise ValueError(f"Image not found or unable to load: {img_path}")
    
    height, width, _ = img.shape

    # Perform OCR to get data dictionary
    data = pytesseract.image_to_data(img, config=config, output_type=Output.DICT)

    horizontal_text = []
    vertical_text = []

    num_boxes = len(data['text'])
    for i in range(num_boxes):
        # Consider only text with confidence > 20
        if int(data['conf'][i]) > 20:
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            detected_text = data['text'][i]

            # Determine horizontal and vertical orientation
            if w > h:  # Wider than tall -> Horizontal
                horizontal_text.append(detected_text)
                color = (0, 255, 0)  # Green for horizontal
            else:  # Taller than wide -> Vertical
                vertical_text.append(detected_text)
                color = (255, 0, 0)  # Blue for vertical

            # Draw bounding boxes
            img = cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

            # Put detected text on the image
            img = cv2.putText(img, detected_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    # Return image, horizontal text, vertical text, and OCR data
    return img, horizontal_text, vertical_text, data